# Info

https://www.kaggle.com/datasets/abedkhooli/arabic-100k-reviews

https://huggingface.co/asafaya/bert-base-arabic

# Libraries

In [1]:
import os
import re

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [4]:
import kagglehub

In [5]:
# !pip install bitsandbytes

### Restart the kernel after installing bitsandbytes

In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import BitsAndBytesConfig

from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding


2026-02-05 14:59:49.118007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770303589.364353      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770303589.431903      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770303590.022115      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770303590.022168      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770303590.022171      24 computation_placer.cc:177] computation placer alr

# Constants

In [7]:
BASE_PATH = "all"
os.makedirs(BASE_PATH, exist_ok=True)

FINE_TUNE_CACHE_PATH = os.path.join(BASE_PATH, "fine_tune_cache")

In [8]:
VAL_SPLIT_RATIO = 0.3

In [9]:
MODEL_NAME = "asafaya/bert-base-arabic"

# Dataset

In [10]:
path = kagglehub.dataset_download("abedkhooli/arabic-100k-reviews")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/arabic-100k-reviews


In [11]:
os.listdir(path)

['ar_reviews_100k.tsv']

In [12]:
df = pd.read_csv(os.path.join(path, "ar_reviews_100k.tsv"), sep='\t')

# Exploration

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   99999 non-null  object
 1   text    99999 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [14]:
df.describe(include='all')

,label,text
count,99999,99999
unique,3,99999
top,Positive,لن ارجع إليه مرة اخرى . قربه من البحر. المكان ...
freq,33333,1


In [15]:
pd.concat(
    [
        df.head(5),
        df.sample(5, random_state=42),
        df.tail(5)
    ]
)

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
26002,Positive,أقم على الماشي . بصورة عامة الغرفة جيدة وكل شي...
80420,Negative,لم تعجبني كباقي السلسلة و غير متحمس لقراءة الج...
19864,Positive,كان المكان ممتاز والأمن والاستقبال اوكي . . ال...
81525,Negative,القصة اجمالا مشوقة كونها تعتمد علي شخصيات يجمع...
57878,Mixed,اربع نجوم لولا الملل الذي اصابني في النهاية......


# Preprocessing

## Nulls

In [16]:
df.isna().sum()

label    0
text     0
dtype: int64

## Duplicates

In [17]:
df.duplicated().sum()

np.int64(0)

## Label

In [18]:
df['label'].value_counts()

label
Positive    33333
Mixed       33333
Negative    33333
Name: count, dtype: int64

In [19]:
def process_label(label):
    label = str(label).strip().lower()

    if label == 'positive':
        return 2
    elif label == 'negative':
        return 0
    else:
        return 1

    return label

In [20]:
df['final_label'] = df['label'].apply(process_label)

In [21]:
df[['label', 'final_label']].sample(10, random_state=42)

,label,final_label
26002,Positive,2
80420,Negative,0
19864,Positive,2
81525,Negative,0
57878,Mixed,1
79451,Negative,0
43083,Mixed,1
80916,Negative,0
60764,Mixed,1
52668,Mixed,1


## Text

In [22]:
def process_text(text):
    # Remove diacritics
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)

    # Keep ONLY Arabic letters + spaces
    text = re.sub(r'[^\u0621-\u064A\s]', '', text)

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [23]:
df['final_text'] = df['text'].apply(process_text)

In [24]:
df[['text', 'final_text']].sample(10)

,text,final_text
56323,اثار فقد طفلها الوحيد بالجنون التام في عقلها. ...,اثار فقد طفلها الوحيد بالجنون التام في عقلها و...
8747,ملاحظاتي حول الرواية:. حبكة تقديم بؤر سرد متعد...,ملاحظاتي حول الرواية حبكة تقديم بؤر سرد متعددة...
55777,فكرة الرواية غريبة وجديدة على كاتب عربي سعودي ...,فكرة الرواية غريبة وجديدة على كاتب عربي سعودي ...
64652,. . التأخير في تسجيل الدخول,التأخير في تسجيل الدخول
24218,كتيب صغير الحجم. . كبير المعني. . قليل الكلام ...,كتيب صغير الحجم كبير المعني قليل الكلام كثير ا...
1471,دعوة حقيقة للضحك ونسيان الهموم على نفس نمط فيل...,دعوة حقيقة للضحك ونسيان الهموم على نفس نمط فيل...
38429,هذه الرواية تتحدث عن الحضور الغائب... لا نشعر ...,هذه الرواية تتحدث عن الحضور الغائب لا نشعر بقي...
87658,ضعيف جدا. . لايوجد به واي فاي وازعاج,ضعيف جدا لايوجد به واي فاي وازعاج
28748,لا أحلام نعلقها علي مشجب الأيام، فنرجوا أن تتح...,لا أحلام نعلقها علي مشجب الأيام فنرجوا أن تتحق...
76142,رواية مراهقة وكذلك الأسلوب واللغة,رواية مراهقة وكذلك الأسلوب واللغة


# Splitting

In [25]:
train_text, test_text, train_label, test_label = train_test_split(
    df['final_text'].tolist(),
    df['final_label'].tolist(),
    test_size=VAL_SPLIT_RATIO,
    random_state=42
)

In [26]:
train_text[12], train_label[12]

('كبداية أعجبت كثيرا بما كتب الفكرة بسيطة وكذلك الأسلوب لكنني شعرت في المنتصف أن الكاتب لم يدر ما الذي يفعله بهذه الرواية فقرر إنهائها بطريقة سريعة متتابعة تقتل فيك كل لحظة للشعور وتمنيت إن كان هناك تعمق أكثر في الوصف خاصة في الأجزاء الأخيرة',
 0)

In [27]:
# Create Hugging Face Dataset from lists
train_dataset = Dataset.from_dict({
    "text": train_text,
    "label": train_label
})

eval_dataset = Dataset.from_dict({
    "text": test_text,
    "label": test_label
})

dataset = DatasetDict({
    "train": train_dataset,
    "validation": eval_dataset
})

In [28]:
print(train_dataset)
print(eval_dataset)
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 69999
})
Dataset({
    features: ['text', 'label'],
    num_rows: 30000
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 69999
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
})


# Model Loading

In [29]:
MODEL_NAME

'asafaya/bert-base-arabic'

In [30]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    # device_map= "auto",
    num_labels= 3 # 3 CLASSES
)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
def preprocess(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/69999 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [32]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 69999
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30000
    })
})

# Finetune

In [33]:
import torch
print(torch.cuda.is_available())  # True if GPU detected
print(torch.cuda.device_count())  # Number of GPUs
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
2
Tesla T4


In [34]:
training_args = TrainingArguments(
    output_dir=FINE_TUNE_CACHE_PATH,
    
    # Saving
    save_strategy="steps",
    save_steps=500,          # save checkpoint every 500 steps
    save_total_limit=2,
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=500,          # evaluate every 500 steps
     
    # Logging
    logging_steps=100,
    
    # Training
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    
    # Best model tracking
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    
    # Hub
    push_to_hub=False,

    # Disable W&B
    report_to=[]
)

In [35]:
data_collator = DataCollatorWithPadding(tokenizer)

In [36]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)  # take highest logit as prediction
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # weighted F1 for multi-class
    return {
        "accuracy": acc,
        "f1": f1
    }

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = data_collator
)

/tmp/ipykernel_24/3601982305.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [38]:
print("START TRAINING...")

START TRAINING...


In [39]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.730900,0.715299,0.676900,0.666415
1000,0.700300,0.681937,0.688600,0.685067
1500,0.650000,0.666980,0.693067,0.695534
2000,0.640600,0.638014,0.711800,0.706889
2500,0.694300,0.636091,0.712167,0.713941
3000,0.652900,0.639422,0.717067,0.714140
3500,0.596900,0.624630,0.722000,0.721528
4000,0.621300,0.619313,0.718633,0.717137
4500,0.520000,0.649317,0.722133,0.721253
5000,0.479400,0.678447,0.726933,0.726954


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=13125, training_loss=0.5182931848435175, metrics={'train_runtime': 26671.2687, 'train_samples_per_second': 7.874, 'train_steps_per_second': 0.492, 'total_flos': 5.525302838262682e+16, 'train_loss': 0.5182931848435175, 'epoch': 3.0})